# **DATA PREPROCESSING BASED ON EDA INSIGHTS**

This notebook implements preprocessing steps based on the comprehensive EDA findings and recommendations. It follows the evidence-based approach from the EDA report to ensure our preprocessing aligns with the data patterns discovered.
Based on the EDA report, I will:
1. **Handled missing values properly**
2. **Handle Skewed Variables** - Application Income, Coapplication Income and Loan amount.
3. **Outlier Treatment** - IQR-capping for extreme Application Income, Coapplication Income and Loan amount
4. **Feature Engineering** - Create and interaction features
    - Total Income (TotalIncome = ApplicantIncome + CoapplicantIncome)
    - Debt-to-Income Ratio (DTI = LoanAmount / TotalIncome)
    - Equated Monthly Instalment Feature (EMI = LoanAmount / Loan_Amount_Term)
5. **Feature Selection** - Keep high-signal features, evaluate low-signal ones
6. **Scaling** - StandardScaler for distance-based models
7. **Target Handling** - Classification approach with stratified splits

In [ ]:
# let call all neccessary libraies
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stat 

In [ ]:
# let load the data from EDA
# load the test data
test_data = pd.read_csv()

## 1. **Handled missing values properly**

In [ ]:
# let check the if missing value still existing


## 2. Handle Skewed Variables

## 3. **Outlier Treatment** - IQR-capping for extreme Application Income, Coapplication Income and Loan amount

## 4. **Feature Engineering** - Create and interaction features

-   Total Income (TotalIncome = ApplicantIncome + CoapplicantIncome)
-   Debt-to-Income Ratio (DTI = LoanAmount / TotalIncome)
-   Equated Monthly Instalment Feature (EMI = LoanAmount / Loan_Amount_Term)